In [ ]:
import sys
sys.path.append("../")

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import os

In [ ]:
import Misc.env_vars as ev
from Logic.base_queries import run_stmt
import Misc.env_vars as ev
from Logic import base_queries as bq

In [ ]:
from Models.models import (od_dq_base, dq_agg_view, dq_agg_sum, dq_col_sum, dq_dim_order_status, dq_dim_sellers, ds_data_sanity, dq_agg_order_stats,
ds_last_run_date)

In [ ]:
from models import engine

In [ ]:
from sqlalchemy import Select, func, or_, desc, and_, asc

## Top Cards 

### Total Orders with Delta

In [ ]:
dt_rng = bq.get_date_range()

max_date = dt_rng[1]

In [ ]:
prev_dt = max_date - timedelta(days=1)

In [ ]:
seller = "agg.dominos.co.in"

In [ ]:
prev_dt

In [ ]:
st_dt = datetime.strptime("2024-08-10","%Y-%m-%d")

In [ ]:
seller = None

In [ ]:
stmt = Select(dq_agg_order_stats).where(and_(
dq_agg_order_stats.c.order_date == max_date,
dq_agg_order_stats.c.seller_np == seller if seller else True)
)

In [ ]:
print(stmt)

In [ ]:
df = pd.DataFrame(run_stmt(stmt))

In [ ]:
df.head(5)